# Mapping Yelp Search Results - Part 1

- 04/26/22


## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [38]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started



> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [40]:
# Load API Credentials
with open('/Users/codingdojo/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [41]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### Define Search Terms and File Paths

In [42]:
# set our API call parameters and filename before the first call
LOCATION = 'Baltimore, MD,21202'
TERM = 'Burgers'

In [43]:
## Specify fodler for saving data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = f"{FOLDER}{TERM}-{LOCATION.split(',')[0]}.json"
JSON_FILE

'Data/Burgers-Baltimore.json'

### Check if Json File exists and Create it if it doesn't

In [44]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Burgers-Baltimore.json already exists.


### Load JSON FIle and account for previous results

In [45]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [46]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results+1)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [47]:
## How many results total?
total_results = results['total']
total_results

532

- Where is the actual data we want to save?

In [48]:
results['businesses']

[{'id': 'AQ56plNP56TIk3JswtoiMA',
  'alias': 'abbey-burger-bistro-baltimore-4',
  'name': 'Abbey Burger Bistro',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nIpyKbeP_5xK5Zf6tQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/abbey-burger-bistro-baltimore-4?adjust_creative=j8tXJtmEvaNIK8mAYJ_C5A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=j8tXJtmEvaNIK8mAYJ_C5A',
  'review_count': 914,
  'categories': [{'alias': 'sportsbars', 'title': 'Sports Bars'},
   {'alias': 'tradamerican', 'title': 'American (Traditional)'}],
  'rating': 4.0,
  'coordinates': {'latitude': 39.2771835822848,
   'longitude': -76.6129885794452},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '1041 Marshall St',
   'address2': '',
   'address3': '',
   'city': 'Baltimore',
   'zip_code': '21230',
   'country': 'US',
   'state': 'MD',
   'display_address': ['1041 Marshall St', 'Baltimore, MD 21230']},
  'phone': '+14434539698',
  'dis

In [49]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [50]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

27

In [51]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location=LOCATION,
                                        term=TERM, 
                                        offset=n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
            
    ## What to do if we get an error/exception.
    except Exception as e: # saving the error message so we can print it.
        print('[!] ERROR: ',e)
        break
        
print('Loop completed.')



  0%|          | 0/27 [00:00<?, ?it/s]

Loop completed.


## Open the Final JSON File

In [52]:
df = pd.read_json(JSON_FILE)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,914,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"{'latitude': 39.2771835822848, 'longitude': -7...",[delivery],$$,"{'address1': '1041 Marshall St', 'address2': '...",+14434539698,(443) 453-9698,2161.212353
1,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media1.fl.yelpcdn.com/bphoto/mmtgGp...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,32,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 39.32696, 'longitude': -76.63753}","[pickup, delivery]",$$,"{'address1': '3300 Clipper Mill Rd', 'address2...",,,4242.702639
2,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,135,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 39.27174, 'longitude': -76.60223}","[pickup, delivery]",$$,"{'address1': '823 E Fort Ave', 'address2': '',...",+14106373699,(410) 637-3699,2754.995467
3,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,266,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 39.2822776123555, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '811 S Broadway', 'address2': '',...",+14105221428,(410) 522-1428,2009.275703
4,WRz58h4VB9nadcyar4LGvQ,wet-city-baltimore,Wet City,https://s3-media4.fl.yelpcdn.com/bphoto/8L30rM...,False,https://www.yelp.com/biz/wet-city-baltimore?ad...,159,"[{'alias': 'beerbar', 'title': 'Beer Bar'}, {'...",4.0,"{'latitude': 39.3016090393066, 'longitude': -7...",[delivery],$$,"{'address1': '223 W Chase St', 'address2': '',...",+14438736699,(443) 873-6699,1143.873380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,vnOzM6USxuhmuXV_uqg0JA,filippos-baltimore,Filippo's,https://s3-media2.fl.yelpcdn.com/bphoto/NiWpYq...,False,https://www.yelp.com/biz/filippos-baltimore?ad...,51,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 39.28699, 'longitude': -76.56748}","[delivery, pickup]",$,"{'address1': '418 S Conkling St', 'address2': ...",+14103274550,(410) 327-4550,3607.859236
527,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,"{'latitude': 39.322087392211, 'longitude': -76...",[delivery],NaN,"{'address1': '3320 Belair Rd', 'address2': '',...",+14105636222,(410) 563-6222,4134.435594
528,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 39.33086, 'longitude': -76.63161}","[delivery, pickup]",$$,"{'address1': '907 W 36th St', 'address2': None...",+14108893663,(410) 889-3663,4382.210159
529,rMrTzM9EET0wszOng1eINA,chelles-kitchen-baltimore,Chelles kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/M4RMzz...,False,https://www.yelp.com/biz/chelles-kitchen-balti...,11,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.5,"{'latitude': 39.32332, 'longitude': -76.57185}","[delivery, pickup]",NaN,"{'address1': '3436 Belair Rd', 'address2': '',...",+14103014414,(410) 301-4414,4311.772603


In [53]:
JSON_FILE

'Data/Burgers-Baltimore.json'

In [54]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Burgers-Baltimore.csv.gz'

In [55]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [61]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))
print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

size_json/size_csv_gz

JSON FILE: 521,412 Bytes
CSV.GZ FILE: 73,453 Bytes


7.098580044382122

## Next Class: Processing the Results and Mapping 